# Introduction

In [ ]:
Introduction to Deep Q-learning (DQN)

Introduction to game we are going to play, e.g. visit website

Complex simulation

Keras etc.

# Practical part

In [46]:
import numpy as np
import random
from collections import deque, namedtuple
import gym

"""
Build networks
"""
class Q_Networks:

    def __init__(self, env):
        pass
    
    def build_Q_network(env):
        pass
    
    def train_q_network(action, state, done, reward, new_state):
        pass
    

class ExperienceReplay:
    
    def __init__(self, buffer_size, batch_size):
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.experience_buffer = deque(maxlen=buffer_size)
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "new_state", "done"])
    
    def add_experience(self, state, action, reward, new_state, done):   
        e = self.experience(state, action, reward, new_state, done)
        self.experience_buffer.append(e)
    
    def get_batch(self):
        if len(self.experience_buffer) < self.batch_size:
            experiences =  self.experience_buffer
        else:
            experiences = random.sample(self.experience_buffer, self.batch_size)
        
        states = np.hstack([e.state for e in experiences if e is not None])
        actions = np.hstack([e.action for e in experiences if e is not None])
        rewards = np.hstack([e.reward for e in experiences if e is not None])
        new_states = np.hstack([e.new_state for e in experiences if e is not None])
        dones = np.hstack([e.done for e in experiences if e is not None])
        
        return (states, actions, rewards, new_states, dones)
    
"""
Explaination of epsilon greedy and action selection
"""
    
def select_action(state, epsilon):
    
    random_value = np.random.uniform()
    if (epsilon > 0.01) and (epsilon > random_value):
        action = env.action_space.sample()
    else:
        action =  np.argmax(Q[state,:])   
    
    return action
        


## Test box

In [47]:
er = ExperienceReplay(10, 2)

er.add_experience(1, 0, 2.0, 1, 0)
er.add_experience(0, -1, 0.0, 1, 1)

sample = er.get_batch()

sample

(array([1, 0]), array([ 0, -1]), array([2., 0.]), array([1, 1]), array([0, 1]))